# Dummy Net

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.data import init_model_logging
import tensorflow as tf
import numpy as np
import pandas as pd
import os

## Animals Data

In [ ]:
# [fur, extinction, weight] 
animals = np.array([[1.0, 0.7, 0.9], # lion
                    [0.8, 0.8, 1.0], # lion
                    [0.1, 0.3, 0.2], # seal
                    [0.0, 0.2, 0.1]  # seal
                   ], dtype=np.float32)

In [ ]:
animal_labels = np.array(
                [[ 1.0],  # lion
                 [ 1.0],  # lion
                 [-1.0],  # seal
                 [-1.0]   # seal
                ], dtype=np.float32)

## Net Without Training 

### Build Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    
    w1 = tf.constant(np.array([[1, 2], [4, 5], [7, 8]], dtype=np.float32), name='w1_const')
    h = tf.matmul(x, w1)

    w2 = tf.constant(np.array([[1], [1]], dtype=np.float32), name='w2_const')
    y = tf.matmul(h, w2)

### Run Session

In [ ]:
session = tf.Session(graph=graph)

In [ ]:
prediction, = session.run([y], feed_dict={x: animals})
print('Prediction on animals:')
print(prediction)

In [ ]:
session.close()

## Net With Loss

### Build Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.constant(np.array([[1, 2], [4, 5], [7, 8]], dtype=np.float32), name='w1_const')
    h = tf.matmul(x, w1)

    w2 = tf.constant(np.array([[1], [1]], dtype=np.float32), name='w2_const')
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)

### Run Session

In [ ]:
session = tf.Session(graph=graph)

In [ ]:
loss_on_animal_data, = session.run([loss], feed_dict={x: animals, label: animal_labels})
print('Loss: {0:.2f}'.format(loss_on_animal_data))

In [ ]:
session.close()

## Net With Trainable Variables

### Build Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
    h = tf.matmul(x, w1)

    w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)
    initialize_vars = tf.global_variables_initializer()

### Run Session

In [ ]:
session = tf.Session(graph=graph)

In [ ]:
session.run(initialize_vars)
w1_values, w2_values = session.run([w1, w2], feed_dict={x: animals, label: animal_labels})
print('w1:')
print(w1_values)
print('w2:')
print(w2_values)

In [ ]:
session.close()

## Net With Train Step

### Build Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
    label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
    h = tf.matmul(x, w1)

    w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
    y = tf.matmul(h, w2)
    
    loss = tf.losses.mean_squared_error(label, y)
    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

    initialize_vars = tf.global_variables_initializer()

### Run Session

In [ ]:
session = tf.Session(graph=graph)
session.run(initialize_vars)

In [ ]:
for iteration in range(10001):
    _, loss_on_animal_data = session.run([train_step, loss], feed_dict={x: animals, label: animal_labels})
    if iteration % 500 == 0:
        print("Iteration {}: loss {}".format(iteration, loss_on_animal_data))

In [ ]:
prediction = session.run(y, feed_dict={x: animals})

In [ ]:
pd.DataFrame({'prediction':prediction[:,0], 'label':animal_labels[:,0]})

In [ ]:
session.close()

## Net with logging

### Function collecting data for logging

In [ ]:
def variable_summaries(name, var):
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))

        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

### Build Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('inputs'):
        x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
        label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    with tf.variable_scope('layer_1'):
        w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
        h = tf.matmul(x, w1)
        variable_summaries('w1', w1)

    with tf.variable_scope('layer_2'):
        w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
        y = tf.matmul(h, w2)
    
    with tf.name_scope('loss'):
        loss = tf.losses.mean_squared_error(label, y)
        variable_summaries('loss', loss)

    with tf.name_scope('training'):
        train_step = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(loss)

    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

### Init Model Logging

In [ ]:
base_dir = '/tensorboard_summaries/dummy_net/'
logging_meta = init_model_logging(base_dir, 'experiment1', graph=graph, remove_existing=True)

In [ ]:
logging_meta

### Run Session

In [ ]:
with tf.Session(graph=graph) as session:
    session.run(initialize_vars)
    
    for iteration in range(10001):
        _, loss_on_animal_data, summaries = session.run(
            [train_step, loss, merge_summaries], feed_dict={x: animals, label: animal_labels})
    
        logging_meta['train_writer'].add_summary(summaries, iteration)
        if iteration % 500 == 0:
            print("Iteration {}: loss {}".format(iteration, loss_on_animal_data))    

    logging_meta['saver'].save(session, logging_meta['model_path'])